# **Two-Sample t-Test**

A **two-sample t-test** checks whether the means of two independent samples are significantly different from each other.

---

## **1. Hypotheses**

**Null Hypothesis (H0):**  
Mean of group 1 = Mean of group 2  
$H_0: \mu_1 = \mu_2$

**Alternative Hypothesis (H1):**  
Means are not equal  
$H_1: \mu_1 \neq \mu_2$

---

## **2. Test Statistic (Equal Variance Assumed)**

The t-statistic is calculated as:

$t = \dfrac{\bar{X}_1 - \bar{X}_2}{S_p \sqrt{\dfrac{1}{n_1} + \dfrac{1}{n_2}}}$

where  

$S_p = \sqrt{\dfrac{(n_1 - 1)S_1^2 + (n_2 - 1)S_2^2}{n_1 + n_2 - 2}}$

---

## **3. Degrees of Freedom**

$df = n_1 + n_2 - 2$

---

## **4. Decision Rule**

If  
$|t_{calculated}| > t_{critical}$  
→ Reject H₀ (significant difference between means)

Otherwise → Fail to reject H₀ (no significant difference)

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('weights_equal.csv')

In [3]:
male_pop = data[data['Gender'] =='Male']
female_pop = data[data['Gender'] =='Female']

In [4]:
female_pop

,Gender,Weight
100,Female,64.197035
101,Female,66.078733
102,Female,75.231055
103,Female,74.272592
104,Female,69.853689
...,...,...
195,Female,66.715770
196,Female,58.008058
197,Female,79.477107
198,Female,69.198221


In [6]:
sample_m = male_pop['Weight'].sample(30)
sample_f = female_pop['Weight'].sample(30)

In [8]:
sample_f.mean()

70.59492215503263

In [9]:
sample_m.mean()

69.41928323309959

In [10]:
from scipy.stats import stats

In [13]:
stats.ttest_ind(sample_m, sample_f, equal_var=True)

C:\Users\bittu\AppData\Local\Temp\ipykernel_15088\480581880.py:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  stats.ttest_ind(sample_m, sample_f, equal_var=True)


TtestResult(statistic=-0.6958378953800914, pvalue=0.489309307432243, df=58.0)

In [14]:
stats.ttest_ind(sample_m, sample_f, equal_var=False)

C:\Users\bittu\AppData\Local\Temp\ipykernel_15088\3161199544.py:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  stats.ttest_ind(sample_m, sample_f, equal_var=False)


TtestResult(statistic=-0.6958378953800916, pvalue=0.4893605882759856, df=56.945597656687156)

In [17]:
import numpy
arr = numpy.array([6, 7, 5, 8])

In [23]:
arr.var()

1.25

In [27]:
for i in range(len(arr)):
    varience =+ (arr[i] - arr.mean()) ** 2
    print((arr[i] - arr.mean()))
varience

-0.5
0.5
-1.5
1.5


2.25

In [26]:
2.25 / 4

0.5625

In [31]:
((arr[0] - arr.mean()) ** 2) + ((arr[1] - arr.mean()) ** 2)  + ((arr[2] - arr.mean()) ** 2) + ((arr[3] - arr.mean()) ** 2 )

5.0

In [35]:
5 / 3

1.6666666666666667

In [33]:
1.6666666666666667 ** 2

2.777777777777778

In [36]:
1.6666666666666667 / 4

0.4166666666666667

In [40]:
((3 * 1.667) + (3 * 0.667)) / 6

1.167

2.0010000000000003